In [1]:
import numpy as np
np.random.seed(0)

SKILLS = ['Plastic', 'Bees', 'CFC', 'Detergents']
QUESTIONS_PER_SKILL = 25

Q_MATRIX = []
for i in range(len(SKILLS)):
    rows = np.zeros((QUESTIONS_PER_SKILL, len(SKILLS)), dtype=int)
    rows[:, i] = 1
    Q_MATRIX.append(rows)

Q_MATRIX = np.vstack(Q_MATRIX) # Skill x Question matrix
Q_MATRIX

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1,

In [2]:
DIFFICULTY = np.random.uniform(0, 1, Q_MATRIX.shape[0]) # Probability of getting a question right for each question

DIFFICULTY

array([0.5488135 , 0.71518937, 0.60276338, 0.54488318, 0.4236548 ,
       0.64589411, 0.43758721, 0.891773  , 0.96366276, 0.38344152,
       0.79172504, 0.52889492, 0.56804456, 0.92559664, 0.07103606,
       0.0871293 , 0.0202184 , 0.83261985, 0.77815675, 0.87001215,
       0.97861834, 0.79915856, 0.46147936, 0.78052918, 0.11827443,
       0.63992102, 0.14335329, 0.94466892, 0.52184832, 0.41466194,
       0.26455561, 0.77423369, 0.45615033, 0.56843395, 0.0187898 ,
       0.6176355 , 0.61209572, 0.616934  , 0.94374808, 0.6818203 ,
       0.3595079 , 0.43703195, 0.6976312 , 0.06022547, 0.66676672,
       0.67063787, 0.21038256, 0.1289263 , 0.31542835, 0.36371077,
       0.57019677, 0.43860151, 0.98837384, 0.10204481, 0.20887676,
       0.16130952, 0.65310833, 0.2532916 , 0.46631077, 0.24442559,
       0.15896958, 0.11037514, 0.65632959, 0.13818295, 0.19658236,
       0.36872517, 0.82099323, 0.09710128, 0.83794491, 0.09609841,
       0.97645947, 0.4686512 , 0.97676109, 0.60484552, 0.73926

In [3]:
STUDENTS = 20
QUESTIONS_PER_USER = 20

logs = []
for i in range(STUDENTS):
    questions = np.random.choice(Q_MATRIX.shape[0], QUESTIONS_PER_USER, replace=False)
    answers = np.random.binomial(1, DIFFICULTY[questions])
    logs.append([(i, questions[j], answers[j]) for j in range(QUESTIONS_PER_USER)])

logs = np.vstack(logs)
logs

array([[ 0, 17,  1],
       [ 0, 68,  1],
       [ 0, 73,  1],
       ...,
       [19, 73,  1],
       [19, 87,  0],
       [19, 89,  1]])

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NeuralCD(nn.Module):
    def __init__(self, q_matrix: np.ndarray, num_students: int, num_layers: int = 3, hidden_size: int = 100):
        super(NeuralCD, self).__init__()
        self.num_students = num_students

        self.Q = nn.Embedding.from_pretrained(
            torch.tensor(q_matrix, dtype=torch.float32), freeze=True)
        self.A = nn.Embedding(num_students, q_matrix.shape[1])
        self.B = nn.Embedding(q_matrix.shape[0], q_matrix.shape[1])
        self.D = nn.Embedding(q_matrix.shape[0], 1)

        activation = nn.Sigmoid()
        self.interaction_function = nn.Sequential()
        self.interaction_function.add_module(
            'linear1', nn.Linear(q_matrix.shape[1], hidden_size))
        self.interaction_function.add_module('activation1', activation)

        for i in range(num_layers - 1):
            self.interaction_function.add_module(
                f'linear{i+2}', nn.Linear(hidden_size, hidden_size))
            self.interaction_function.add_module(
                f'activation{i+2}', activation)
        self.interaction_function.add_module(
            'linear_final', nn.Linear(hidden_size, 1))
        self.interaction_function.add_module('sigmoid', nn.Sigmoid())

    def forward(self, student_id: torch.Tensor, question_id: torch.Tensor):
        h_s: torch.Tensor = torch.sigmoid(self.A(student_id))
        Q_e: torch.Tensor = self.Q(question_id)
        h_diff: torch.Tensor = torch.sigmoid(self.B(question_id))
        h_disc: torch.Tensor = torch.sigmoid(self.D(question_id))

        x = Q_e * (h_s - h_diff) * h_disc
        x = self.interaction_function(x)
        return x.view(-1)
    
    def get_user_embedding(self, student_id):
        i = torch.tensor(student_id, dtype=torch.long).to(DEVICE)
        return torch.sigmoid(self.A(i)).detach().cpu().numpy()

    def predict(self, student_id: torch.Tensor, question_id: torch.Tensor):
        self.eval()
        with torch.no_grad():
            return self(student_id, question_id).detach().cpu().numpy()

    def fit(self, dataset: Dataset, epochs: int = 10, batch_size: int = 32, lr: float = 0.001, weight_decay: float = 0.0):
        self.train()
        optimizer = optim.Adam(self.parameters(), lr=lr,
                               weight_decay=weight_decay)
        criterion = nn.BCELoss()

        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        for _ in (bar := tqdm(range(epochs))):
            for student_id, question_id, answer in dataloader:
                optimizer.zero_grad()
                student_id, question_id, answer = student_id.to(
                    DEVICE), question_id.to(DEVICE), answer.to(DEVICE)
                y_pred = self(student_id, question_id)
                loss = criterion(y_pred, answer)
                loss.backward()
                optimizer.step()
            bar.set_postfix(loss=loss.item())


class CDataset(Dataset):
    def __init__(self, logs: np.ndarray):
        self.logs = logs

    def __len__(self):
        return len(self.logs)

    def __getitem__(self, idx):
        x = torch.tensor(self.logs[idx, :], dtype=torch.long)
        return x[0], x[1], x[2].float()

In [5]:
model = NeuralCD(Q_MATRIX, STUDENTS).to(DEVICE)
dataset = CDataset(logs)
model.fit(dataset, lr=1e-3, epochs=100, batch_size=16)

  0%|          | 0/100 [00:00<?, ?it/s]